In [70]:
import string

import json
from lxml import html
import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment
import re

from urllib.request import urlopen

from html.parser import HTMLParser
import html2text
from dateutil.parser import parse as parse_date
from urllib.parse import urlparse
import os
from bs4 import SoupStrainer

If want to directly scrape url links from web page with desired links

In [272]:
# search_url = 'http://infoweb.newsbank.com/resources/search/nb?p=WORLDNEWS&b=results&action=search&t=pubname%3ASTIW%21Seattle%2BTimes%252C%2BThe%2B%2528WA%2529&fld0=alltext&val0=fremont+murray+lenin&bln1=AND&fld1=YMD_date&val1=&sort=_rank_%3AD'

# parsed_search_url = urlparse(search_url)
# root_url = parsed_search_url.scheme + "://" + parsed_search_url.hostname

# search_string = 'resources/doc/nb/news'

# page = requests.get(search_url)

# url_list = []

# for link in BeautifulSoup(response.content, 'html.parser', parse_only=SoupStrainer('a')):
#     if link.has_attr('href'):
#         if search_string in link['href']:
#             full_path = root_url + link['href']
#             url_list.append(full_path)

If want to read in url links from a text file

In [275]:
url_list_filename = 'seattle_times_quake_article_urls.txt'

with open(url_list_filename, "rt") as f:
    url_list = [url.strip() for url in f.readlines()]

In [285]:
pages = [requests.get(url).text for url in url_list]
soup_pages = [BeautifulSoup(page, "lxml") for page in pages]

Scrape articles form list of url links

In [301]:
articles = []
for (soup_page, url) in zip(soup_pages, url_list):

    article = soup_page.find('div', {'class': 'body'}).text.strip()
    article = article.replace('\n', ' ')

    try:
        author = soup_page.find('li', {'class': 'author first'}).find_next().find_next().text.strip().title()
        if ',' in author:
            author = author.replace(re.search(r"(,.*)", author, re.I | re.M | re.S).groups()[0], '')
        
    except:
        author = 'Unknown'
    
    try:
        edition = soup_page.find('li', {'class': 'edition'}).find_next().find_next().text.strip().title()
    except:
        edition = 'Unknown'
   
    try:
        section = soup_page.find('li', {'class': 'section'}).find_next().find_next().text.strip().title()
        
        if section.lower() == 'local':
            section = 'Local News'
    except:
        section = 'Unknown'

    try:
        page = soup_page.find('li', {'class': 'page'}).find_next().find_next().text.strip()
    except:
        page = 'Unknown'

    try:
        column = soup_page.find('li', {'class': 'column'}).find_next().find_next().text.strip().title()
    except:
        column = 'Unknown'
    
    try:
        title = soup_page.find('div', {'class': 'title'}).text.strip()
        
        if title == title.upper():
            title = title.capitalize()
    
        if '/' in title:
            title = title.replace('/', '-')
   
    except:
        title = 'Unknown'

    try:
        date = soup_page.find('div', {'class': 'source'}).text.strip()
        
        if 'Seattle Times, The (WA) - ' in date:
            date = date.replace('Seattle Times, The (WA) - ', '')
        if 'Seattle Times, The (WA) (Published as THE SEATTLE TIMES)  - ' in date:
            date = date.replace('Seattle Times, The (WA) (Published as THE SEATTLE TIMES)  - ', '')
        if 'Browse Issues' in date:
            date = date.replace('Browse Issues', '')
        
        date = parse_date(date).strftime('%Y-%m-%d')
    except:
        date = 'Unknown'

    folder = 'SeattleTimesQuakeArticles/json/'
    filename = (date + '_' + 
        title[:100].title().replace(' ', '').replace(',', '').replace("'", "").replace('?', '').replace('.', '').replace(';', '').replace(':', '') + 
        '.json')
    file_path = folder + filename

    article_dict = {
                'title' : title,
                'date' : date,
                'author' : author,
                'edition': edition,
                'section' : section,
                'page' : page,
                'text' : article,
                'filename' : filename,
                'url' : url
            }

    articles.append(article_dict)

    with open(file_path, 'w') as f:
        json.dump(article_dict, f)

Print out metadata about a few of the articles

In [298]:
print('Number of articles: ', len(articles))
print('-----------------------------', '\n')
for art in articles[:5]:
    print(art['title'])
    print(art['date'])
    print(art['author'])
    print(art['section'])
    print(art['page'])
    print(art['filename'])
    print(art['url'])
    print(art['text'][:200])
    print('========================================', '\n')

Number of articles:  2079
----------------------------- 

Fund earthquake
2017-06-05
Seattle Times Editorial Board
Editorial
A13
2017-06-05_FundEarthquake.json
http://infoweb.newsbank.com/resources/doc/nb/news/164D41CE27B17408?p=WORLDNEWS
America provides for the common defense of its people in many ways, and not all involve the military. One inexpensive way to protect millions of citizens is to warn them of major earthquakes, which ar

Earthquake early-alert system in jeopardy
2017-05-27
Rong-Gong Lin Ii
News
A1
2017-05-27_EarthquakeEarly-AlertSystemInJeopardy.json
http://infoweb.newsbank.com/resources/doc/nb/news/164BCEAC4C0898D0?p=WORLDNEWS
LOS ANGELES — President Donald Trump’s budget proposal would cut federal funding for an earthquake early-warning system for California, Oregon and Washington state, a development that seismology exper

Ride out a fake  quake, typhoon: Theme parks give jolt of urgency
2017-05-15
Sandi Doughton
Local News
A1
2017-05-15_RideOutAFakeQuakeTyphoonTheme

Test to make sure can read in a generated json file

In [268]:
with open(articles[0]['filename'], 'r') as f:
     test_json_file = json.load(f)

test_json_file['title']
print(test_json_file['title'])
print(test_json_file['date'])
print(test_json_file['author'])
print(test_json_file['section'])
print(test_json_file['page'])
print(test_json_file['filename'])
print(test_json_file['url'])
print(test_json_file['text'][:200])

Murray calls for removal of monument and statue
2017-08-18
Paige Cornwell
Local News
B1
2017-08-18_MurrayCallsForRemovalOfMonumentAndStatue.json
http://infoweb.newsbank.com/resources/doc/nb/news/1665A81FD98D65A8?p=WORLDNEWS
Seattle Mayor Ed Murray has called for both the monument to Confederate soldiers at Capitol Hill’s Lake View Cemetery and a statue of Vladimir Lenin in Fremont to be taken down, saying they represent 
